In [ ]:
import os
import operator
from typing import TypedDict, Annotated, List, Optional
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.postgres import PostgresSaver
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from pydantic import BaseModel, Field
import re

In [2]:
load_dotenv('api_secret.env')
api_key = os.environ.get('API_KEY')


In [3]:
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash', google_api_key=api_key)

In [ ]:
class figures(BaseModel):
    amount : int = Field(description='What is the amount the user is looking for?')
    interest : float = Field(description='Always suggest interest rate above 10 percent')
    tenure : int = Field(description='Should always be greater than 1 year')


In [48]:
structured_llm = llm.with_structured_output(figures)

In [ ]:
#tools are here 



In [ ]:
class Loan_agent_state(TypedDict):
    #contains message history of both human and agent
    messages : Annotated[List[BaseMessage], operator.add]

    customer_phone : int
    user_details : Optional[dict]
    credit_score : Optional[int]
    is_verified : bool

    needs_income_verification : bool
    is_income_verified : bool

    sanction_letter_path : Optional[str]

    routing_decision : str

In [ ]:
# SalesAgent node, this node communicated with the user and learn intent. 
def SalesAgent(state: Loan_agent_state) :
    prompt = "Greet the user, welcoming them to Tata Capital Personal Loan Services"
    
    #check 1
    if len(state['messages']) == 1:
        ai_response = llm.invoke(prompt).content
        return {'messages': [AIMessage(content=ai_response)], 'routing_decision':'waiting_for_user'}
    
    #check2
    elif len(state['messages']) > 1:
        last_message = state['messages'][-1].content
        last_message_obj = state['messages'][-1]

        # checking if the customer has been presented with loan options
        # also checking if the customer has selected a loan or not

        if state.get('sanction_letter_path') and not isinstance(last_message_obj, HumanMessage):
            print("---LOGIC: Presenting final sanction letter---")
            
            customer_name = state.get('customer_details', {}).get('name', 'Customer')
            letter_path = state.get('sanction_letter_path')
            
            final_message = (
                f"Congratulations, {customer_name}! Your loan has been approved. \n\n"
                f"You can download your sanction letter here: {letter_path}\n\n"
                "Thank you for choosing Tata Capital!"
            )
            
            return {
                'messages': [AIMessage(content=final_message)],
                'routing_decision': 'end_conversation' # This tells the graph to stop
            }
        
        if state.get('presented_options') and not state.get('selected_loan'):
            options_list = state['presented_options']
            
            
            # CHECK 5: The user has replied to the offers.
            if isinstance(last_message_obj, HumanMessage):
                print("---LOGIC: User is making a selection. Handing off to extractor.---")
                return {
                    "routing_decision": "goto_extraction"
                }
            else:
                print("---LOGIC: Presenting loan offers---")
                
                options_list = state['presented_options']
                formatted_options = []
                for i, option in enumerate(options_list):
                    option_str = (
                        f"  {i+1}. A loan of ₹{option.amount:,.0f} "
                        f"at {option.interest_rate:.1f}% "
                        f"for {option.tenure_years} years."
                    )
                    formatted_options.append(option_str)
                
                customer_name = state.get('customer_details', {}).get('name', 'there')
                
                final_message = (
                    f"Great news, {customer_name}! "
                    f"Based on your profile, I've found these options for you:\n\n"
                    f"{'\n'.join(formatted_options)}\n\n"
                    "Please let me know which option you'd like to proceed with (e.g., 'option 1')."
                )
                
                return {
                    'messages': [AIMessage(content=final_message)],
                    'routing_decision': 'waiting_for_user'
                } 
            
        
        #check for phone no if given
        match = re.search(r'\b(\d{10})\b', last_message)

        #if match found we check if they are not verified
        if match and (not state.get('is_verified')):
            ph_no = match.group(1)
            return {'customer_phone': ph_no, 'routing_decision':'goto_verification'}
        
        # if the last message was not phone number, what was it about? 
        # Checking intent in this part of statements

        #if the llm will respond with 'yes' or 'no'
        intent = llm.invoke(f"Does this user want to start a loan application? Respond with 'yes' or 'no': '{last_message}").content.lower().strip()
        
        #intent verified to be 'yes'
        if intent == "yes" and (not state.get('is_verified')) :
            ask_ph_no = llm.invoke("You are a friendly bank agent. The user wants a loan. Ask them for their 10-digit phone number to begin verification.") 
            return {'messages': [AIMessage(content=ask_ph_no)], 'routing_decision':'waiting_for_user'}
        else:
            general_response = llm.invoke(state['messages']).content
        
            return {
                'messages': [AIMessage(content=general_response)],
                'routing_decision': 'wait_for_user'
            }
        

        
    





In [ ]:
def Master(state: Loan_agent_state):
    
    # general chat 
    directive = SystemMessage(content="You are a friendly and conversational banking assistant whose goal is to understand a customer's intentions behind exploring a personal loan — without discussing numbers or eligibility. " \
    "Greet the user warmly, ask simple and natural questions to learn what they need the loan for (like travel, education, emergencies, or lifestyle goals), and respond with empathy and encouragement. " \
    "Keep the tone approachable, positive, and human — like a helpful friend guiding them through their options. " \
    "Focus only on understanding their purpose and building trust, not selling or quoting figures."\
    "However if the user already wants to get a loan and already has his own figures like amount, rate and interest etc, then negotiate the numbers with him based on the following context : You are a highly skilled and persuasive financial advisor working for a leading bank. " \
    "Your mission is to understand each customer's unique financial situation, identify their core needs, and confidently guide them toward taking a personal loan that best aligns with their goals. " \
    "Communicate with authority, empathy, and clarity — ask insightful questions to uncover motivations such as education expenses, medical emergencies, debt consolidation, or lifestyle improvements. " \
    "Once you understand their priorities, negotiate terms strategically, address concerns with transparency, and demonstrate how the personal loan can genuinely improve their financial well-being through flexibility, better cash flow, or responsible credit building. " \
    "Maintain a consultative yet assertive tone — professional, approachable, and persuasive — ensuring the customer feels understood and empowered, not pressured. " \
    "Use emotional intelligence and data-driven reasoning to build trust, overcome objections, and clearly present the loan as a smart, beneficial decision. " \
    "Conclude every conversation with confidence, guiding the customer toward an actionable next step such as completing the loan application, ensuring they view the offer as a pathway to financial empowerment and peace of mind.")

    response = llm.invoke([directive, HumanMessage(content=state['user_message'])]).content
    state['message'].append(response)
    
    return state

In [ ]:
def Sales(state: Loan_agent_state) -> Loan_agent_state: 
    
    directive = SystemMessage(content="You are a highly skilled and persuasive financial advisor working for a leading bank. " \
    "Your mission is to understand each customer's unique financial situation, identify their core needs, and confidently guide them toward taking a personal loan that best aligns with their goals. " \
    "Communicate with authority, empathy, and clarity — ask insightful questions to uncover motivations such as education expenses, medical emergencies, debt consolidation, or lifestyle improvements. " \
    "Once you understand their priorities, negotiate terms strategically, address concerns with transparency, and demonstrate how the personal loan can genuinely improve their financial well-being through flexibility, better cash flow, or responsible credit building. " \
    "Maintain a consultative yet assertive tone — professional, approachable, and persuasive — ensuring the customer feels understood and empowered, not pressured. " \
    "Use emotional intelligence and data-driven reasoning to build trust, overcome objections, and clearly present the loan as a smart, beneficial decision. " \
    "Conclude every conversation with confidence, guiding the customer toward an actionable next step such as completing the loan application, ensuring they view the offer as a pathway to financial empowerment and peace of mind.")

    response = structured_llm.invoke([directive, HumanMessage(content=state['user_message'])])

    state['amount'] = response

    return state
    



In [ ]:
def Verification(state: Loan_agent_state):
    


    

In [25]:
def UnderWriting(state: Loan_agent_state):
    return 0

In [26]:
def Sanction_letter_generate(state: Loan_agent_state):
    return 0

In [ ]:
def Routing(state: Loan_agent_state):
    if state['user_message'] == 'exit':
        return 'end'
    else:
        return 0
        

In [29]:
graph = StateGraph(Loan_agent_state)

#adding nodes
graph.add_node('Master', Master)
graph.add_node('Sales', Sales)
graph.add_node('Verification', Verification)
graph.add_node('UnderWriting', UnderWriting)
graph.add_node('Sanction_letter_generate', Sanction_letter_generate)

#adding edges
graph.add_edge(START, 'Master')
graph.add_edge('Master','Sales')
graph.add_edge('Sales','Verification')
graph.add_edge('Verification','UnderWriting')
graph.add_edge('UnderWriting','Sanction_letter_generate')
graph.add_edge('Sanction_letter_generate',END)

#compiling the graph

app = graph.compile()



